# All PPTs

In [2]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [4]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 4.6 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
!pip install pyLDAvis==3.2.1

In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
all = pd.read_csv('Usability Test Data.csv')
all = all.loc[all['Person'] != 'Interviewer']

In [9]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(all['Question/Answer']))

print(data_words[:1])

[['well', 'mean', 'my', 'last', 'trip', 'was', 'mostly', 'because', 'already', 'had', 'flight', 'that', 'couldn', 'use', 'which', 'moved', 'to', 'next', 'year', 'but', 'then', 'was', 'like', 'well', 'it', 'doesn', 'matter', 'so', 'thought', 'll', 'just', 'come', 'to', 'berlin', 'because', 'have', 'friends', 'here', 'and', 'that', 'was', 'through', 'easyjet', 'and', 'mean', 'had', 'to', 'do', 'few', 'things', 'on', 'the', 'website', 'to', 'change', 'the', 'flights', 'and', 'was', 'charged', 'bit', 'of', 'money', 'but', 'think', 'yeah', 'there', 'wasn', 'much', 'planning', 'in', 'it', 'because', 'it', 'was', 'more', 'of', 'kind', 'of', 'like', 'improvised', 'trip', 'what', 'was', 'the', 'other', 'part', 'of', 'the', 'question']]


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['well', 'mean', 'my', 'last', 'trip', 'was', 'mostly', 'because', 'already', 'had', 'flight', 'that', 'couldn', 'use', 'which', 'moved', 'to', 'next', 'year', 'but', 'then', 'was', 'like', 'well', 'it', 'doesn', 'matter', 'so', 'thought', 'll', 'just', 'come', 'to', 'berlin', 'because', 'have', 'friends', 'here', 'and', 'that', 'was', 'through', 'easyjet', 'and', 'mean', 'had', 'to', 'do', 'few', 'things', 'on', 'the', 'website', 'to', 'change', 'the', 'flights', 'and', 'was', 'charged', 'bit', 'of', 'money', 'but', 'think', 'yeah', 'there', 'wasn', 'much', 'planning', 'in', 'it', 'because', 'it', 'was', 'more', 'of', 'kind', 'of', 'like', 'improvised', 'trip', 'what', 'was', 'the', 'other', 'part', 'of', 'the', 'question']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['mean', 'last', 'trip', 'mostly', 'already', 'flight', 'move', 'next', 'year', 'well', 'matter', 'thought', 'come', 'friend', 'easyjet', 'mean', 'thing', 'website', 'change', 'flight', 'charge', 'bit', 'money', 'think', 'much', 'planning', 'kind', 'improvise', 'trip', 'part', 'question']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1)]]


In [14]:
id2word[0]

'already'

In [15]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('already', 1),
  ('bit', 1),
  ('change', 1),
  ('charge', 1),
  ('come', 1),
  ('easyjet', 1),
  ('flight', 2),
  ('friend', 1),
  ('improvise', 1),
  ('kind', 1),
  ('last', 1),
  ('matter', 1),
  ('mean', 2),
  ('money', 1),
  ('mostly', 1),
  ('move', 1),
  ('much', 1),
  ('next', 1),
  ('part', 1),
  ('planning', 1),
  ('question', 1),
  ('thing', 1),
  ('think', 1),
  ('thought', 1),
  ('trip', 2),
  ('website', 1),
  ('well', 1),
  ('year', 1)]]

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.082*"open" + 0.049*"put" + 0.034*"pick" + 0.032*"want" + 0.029*"price" + '
  '0.027*"know" + 0.024*"give" + 0.022*"super" + 0.022*"bad" + '
  '0.022*"sometimes"'),
 (1,
  '0.087*"early" + 0.062*"late" + 0.056*"flight" + 0.055*"browse" + '
  '0.046*"usually" + 0.041*"return" + 0.037*"price" + 0.030*"thing" + '
  '0.029*"whole" + 0.026*"mean"'),
 (2,
  '0.104*"inaudible" + 0.097*"date" + 0.081*"ok" + 0.068*"departure" + '
  '0.055*"good" + 0.045*"never" + 0.043*"later" + 0.038*"think" + '
  '0.030*"refund" + 0.023*"use"'),
 (3,
  '0.074*"mess" + 0.043*"list" + 0.029*"information" + 0.015*"way" + '
  '0.015*"complex" + 0.015*"manner" + 0.015*"layout" + 0.015*"basically" + '
  '0.015*"suspicious" + 0.015*"main"'),
 (4,
  '0.109*"actual" + 0.054*"air" + 0.054*"luggage" + 0.047*"cabin" + 0.042*"go" '
  '+ 0.026*"still" + 0.023*"exist" + 0.023*"skyscanner" + 0.017*"browse" + '
  '0.012*"good"'),
 (5,
  '0.019*"piss" + 0.019*"money" + 0.019*"do" + 0.015*"alright" + 0.007*"fuck" '
  '

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.073749250432356

Coherence Score:  0.4455539033523067


In [20]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.240531 -0.157359       1        1  26.902918
6      0.339435  0.000265       2        1  10.850128
17    -0.243157 -0.251686       3        1  10.492452
1     -0.338655 -0.031498       4        1   7.523334
16     0.178609 -0.258075       5        1   6.518847
0     -0.124964 -0.173503       6        1   5.707802
7     -0.068543 -0.338599       7        1   5.577123
12     0.160695  0.214272       8        1   5.085032
2      0.203629  0.309087       9        1   4.906736
19     0.159461 -0.056663      10        1   3.911056
14     0.024544 -0.194848      11        1   2.637730
10    -0.094953  0.283728      12        1   2.388547
4      0.110000  0.098555      13        1   2.143272
15    -0.214151  0.139897      14        1   1.737145
8     -0.051084  0.166212      15        1   1.260721
13    -0.108091  0.073798      16        1   0.685566
3     -0.070574  0.017533      17        1   0.602550
9     -0.047976  0.071091      18        1   0.582756
11    -0.020120  0.044725      19        1   0.270718
5     -0.034635  0.043068      20        1   0.215570, topic_info=        Term       Freq      Total Category  logprob  loglift
47      know  66.000000  66.000000  Default  30.0000  30.0000
44       get  31.000000  31.000000  Default  29.0000  29.0000
107    click  26.000000  26.000000  Default  28.0000  28.0000
259  airline  34.000000  34.000000  Default  27.0000  27.0000
22     think  52.000000  52.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
373  ideally   0.009805   1.159425  Topic20  -6.4424   1.3669
374   length   0.009805   1.159425  Topic20  -6.4424   1.3668
372      hmm   0.009805  11.632572  Topic20  -6.4424  -0.9390
375   period   0.009805   1.159426  Topic20  -6.4424   1.3668
212      try   0.009805  12.273940  Topic20  -6.4424  -0.9927

[950 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
31        4  0.832026  accessibility
475       6  0.755871        account
240      13  0.952435         actual
32        1  0.451863       actually
32        4  0.338898       actually
...     ...       ...            ...
98        6  0.222083           work
98       16  0.444165           work
27        1  0.245833           year
27        3  0.491665           year
27        6  0.245833           year

[691 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 7, 18, 2, 17, 1, 8, 13, 3, 20, 15, 11, 5, 16, 9, 14, 4, 10, 12, 6])

In [19]:
pyLDAvis.save_html(vis, 'All PPTs.html')

# PPT01

In [20]:
all = pd.read_csv('Usability Test Data.csv')
PPT01 = all.loc[(all['Person'] == 'PPT01') & (all['Person'] != 'Interviewer')]

In [21]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT01['Question/Answer']))

print(data_words[:1])

[['well', 'mean', 'my', 'last', 'trip', 'was', 'mostly', 'because', 'already', 'had', 'flight', 'that', 'couldn', 'use', 'which', 'moved', 'to', 'next', 'year', 'but', 'then', 'was', 'like', 'well', 'it', 'doesn', 'matter', 'so', 'thought', 'll', 'just', 'come', 'to', 'berlin', 'because', 'have', 'friends', 'here', 'and', 'that', 'was', 'through', 'easyjet', 'and', 'mean', 'had', 'to', 'do', 'few', 'things', 'on', 'the', 'website', 'to', 'change', 'the', 'flights', 'and', 'was', 'charged', 'bit', 'of', 'money', 'but', 'think', 'yeah', 'there', 'wasn', 'much', 'planning', 'in', 'it', 'because', 'it', 'was', 'more', 'of', 'kind', 'of', 'like', 'improvised', 'trip', 'what', 'was', 'the', 'other', 'part', 'of', 'the', 'question']]


In [22]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['well', 'mean', 'my', 'last', 'trip', 'was', 'mostly', 'because', 'already', 'had', 'flight', 'that', 'couldn', 'use', 'which', 'moved', 'to', 'next', 'year', 'but', 'then', 'was', 'like', 'well', 'it', 'doesn', 'matter', 'so', 'thought', 'll', 'just', 'come', 'to', 'berlin', 'because', 'have', 'friends', 'here', 'and', 'that', 'was', 'through', 'easyjet', 'and', 'mean', 'had', 'to', 'do', 'few', 'things', 'on', 'the', 'website', 'to', 'change', 'the', 'flights', 'and', 'was', 'charged', 'bit', 'of', 'money', 'but', 'think', 'yeah', 'there', 'wasn', 'much', 'planning', 'in', 'it', 'because', 'it', 'was', 'more', 'of', 'kind', 'of', 'like', 'improvised', 'trip', 'what', 'was', 'the', 'other', 'part', 'of', 'the', 'question']


In [23]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['mean', 'last', 'trip', 'mostly', 'already', 'flight', 'move', 'next', 'year', 'well', 'matter', 'thought', 'come', 'friend', 'easyjet', 'mean', 'thing', 'website', 'change', 'flight', 'charge', 'bit', 'money', 'think', 'much', 'planning', 'kind', 'improvise', 'trip', 'part', 'question']]


In [25]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1)]]


In [26]:
id2word[0]

'already'

In [27]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('already', 1),
  ('bit', 1),
  ('change', 1),
  ('charge', 1),
  ('come', 1),
  ('easyjet', 1),
  ('flight', 2),
  ('friend', 1),
  ('improvise', 1),
  ('kind', 1),
  ('last', 1),
  ('matter', 1),
  ('mean', 2),
  ('money', 1),
  ('mostly', 1),
  ('move', 1),
  ('much', 1),
  ('next', 1),
  ('part', 1),
  ('planning', 1),
  ('question', 1),
  ('thing', 1),
  ('think', 1),
  ('thought', 1),
  ('trip', 2),
  ('website', 1),
  ('well', 1),
  ('year', 1)]]

In [28]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [29]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.091*"go" + 0.037*"mean" + 0.037*"want" + 0.037*"ahead" + 0.037*"usually" '
  '+ 0.037*"month" + 0.019*"pick" + 0.019*"orly" + 0.019*"browse" + '
  '0.019*"trip"'),
 (1,
  '0.093*"website" + 0.047*"easyjet" + 0.047*"work" + 0.024*"specific" + '
  '0.024*"ninety" + 0.024*"say" + 0.024*"step" + 0.024*"sort" + 0.024*"part" + '
  '0.024*"difficult"'),
 (2,
  '0.064*"section" + 0.043*"explore" + 0.043*"come" + 0.043*"go" + '
  '0.043*"maybe" + 0.043*"want" + 0.022*"range" + 0.022*"make" + 0.022*"bunch" '
  '+ 0.022*"place"'),
 (3,
  '0.046*"go" + 0.046*"ohh" + 0.039*"know" + 0.031*"flight" + 0.031*"thing" + '
  '0.031*"let" + 0.031*"price" + 0.023*"see" + 0.023*"want" + 0.016*"think"'),
 (4,
  '0.081*"flight" + 0.045*"time" + 0.030*"go" + 0.030*"care" + 0.030*"hour" + '
  '0.030*"bottom" + 0.030*"link" + 0.029*"much" + 0.028*"kind" + 0.025*"give"'),
 (5,
  '0.035*"flight" + 0.035*"mean" + 0.035*"trip" + 0.035*"money" + 0.018*"seem" '
  '+ 0.018*"go" + 0.018*"fuck" + 0.018*"price" +

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.070447459425245

Coherence Score:  0.6090200479853406


In [31]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.188219 -0.137266       1        1  13.195084
7     -0.264157 -0.028726       2        1  11.882611
10    -0.047210 -0.201122       3        1   9.020248
13    -0.081407 -0.249861       4        1   6.771875
4      0.013999  0.261350       5        1   6.072737
19    -0.209614  0.099931       6        1   6.044207
14    -0.126114  0.223918       7        1   5.074798
15    -0.081667 -0.110885       8        1   5.017507
5      0.078947 -0.203390       9        1   5.015908
12     0.251197 -0.069293      10        1   4.845927
0      0.161407 -0.164459      11        1   4.621094
9      0.234980  0.104601      12        1   4.451757
11    -0.184598  0.027940      13        1   3.927820
2     -0.092365  0.061948      14        1   3.813518
17     0.014798  0.166072      15        1   3.534126
1      0.167665  0.182910      16        1   3.356562
16     0.186896  0.012334      17        1   1.764227
18     0.087635  0.005847      18        1   1.301489
8      0.044012  0.007970      19        1   0.184583
6      0.033814  0.010181      20        1   0.103923, topic_info=           Term       Freq      Total Category  logprob  loglift
21        thing  16.000000  16.000000  Default    30.00  30.0000
25      website  11.000000  11.000000  Default    29.00  29.0000
6        flight  24.000000  24.000000  Default    28.00  28.0000
104  skyscanner   7.000000   7.000000  Default    27.00  27.0000
156        seem   9.000000   9.000000  Default    26.00  26.0000
..          ...        ...        ...      ...      ...      ...
179      spammy   0.003104   1.547862  Topic20    -5.68   0.6574
154      return   0.003104   4.699356  Topic20    -5.68  -0.4532
58          see   0.003104  14.684667  Topic20    -5.68  -1.5926
6        flight   0.003104  24.985075  Topic20    -5.68  -2.1241
151         ohh   0.003104   8.498169  Topic20    -5.68  -1.0456

[849 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
31       10  0.707432  accessibility
240       4  0.686538         actual
32        1  0.370748       actually
32        4  0.185374       actually
32        7  0.185374       actually
...     ...       ...            ...
98        7  0.287162           work
98       12  0.287162           work
98       16  0.287162           work
27        9  0.467360           year
27       11  0.467360           year

[651 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 11, 14, 5, 20, 15, 16, 6, 13, 1, 10, 12, 3, 18, 2, 17, 19, 9, 7])

In [32]:
pyLDAvis.save_html(vis, 'PPT01.html')

# PPT02

In [33]:
all = pd.read_csv('Usability Test Data.csv')
PPT02 = all.loc[(all['Person'] == 'PPT02') & (all['Person'] != 'Interviewer')]

In [34]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT02['Question/Answer']))

print(data_words[:1])

[['actually', 'planned', 'trip', 'last', 'week', 'for', 'forthcoming', 'trip', 'during', 'nye', 'mean', 'it', 'was', 'quite', 'an', 'is', 'it', 'fine', 'telling', 'you', 'about', 'forthcoming', 'trip']]


In [35]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['actually', 'planned', 'trip', 'last', 'week', 'for', 'forthcoming', 'trip', 'during', 'nye', 'mean', 'it', 'was', 'quite', 'an', 'is', 'it', 'fine', 'telling', 'you', 'about', 'forthcoming', 'trip']


In [36]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [37]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['actually', 'plan', 'trip', 'last', 'week', 'forthcoming', 'trip', 'nye', 'mean', 'quite', 'fine', 'tell', 'forthcoming', 'trip']]


In [38]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 3), (10, 1)]]


In [39]:
id2word[0]

'actually'

In [40]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('actually', 1),
  ('fine', 1),
  ('forthcoming', 2),
  ('last', 1),
  ('mean', 1),
  ('nye', 1),
  ('plan', 1),
  ('quite', 1),
  ('tell', 1),
  ('trip', 3),
  ('week', 1)]]

In [41]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [42]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.057*"think" + 0.043*"skyscanner" + 0.043*"right" + 0.029*"flight" + '
  '0.029*"share" + 0.029*"usually" + 0.029*"help" + 0.029*"good" + '
  '0.029*"date" + 0.015*"want"'),
 (1,
  '0.036*"know" + 0.036*"let" + 0.036*"honest" + 0.036*"air" + 0.036*"ryanair" '
  '+ 0.036*"think" + 0.036*"go" + 0.019*"friendly" + 0.019*"tell" + '
  '0.019*"faith"'),
 (2,
  '0.049*"cheap" + 0.044*"refund" + 0.039*"option" + 0.038*"think" + '
  '0.033*"honest" + 0.033*"give" + 0.030*"really" + 0.027*"sure" + 0.027*"go" '
  '+ 0.022*"know"'),
 (3,
  '0.100*"way" + 0.050*"fill" + 0.050*"let" + 0.050*"round" + 0.026*"flight" + '
  '0.026*"look" + 0.026*"barcelona" + 0.026*"late" + 0.026*"include" + '
  '0.026*"back"'),
 (4,
  '0.079*"think" + 0.053*"interface" + 0.027*"come" + 0.027*"good" + '
  '0.027*"help" + 0.027*"friendly" + 0.027*"know" + 0.027*"used" + '
  '0.027*"user" + 0.027*"usually"'),
 (5,
  '0.068*"cheap" + 0.068*"go" + 0.057*"flight" + 0.034*"time" + '
  '0.034*"trigger" + 0.023*"reall

In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.029150370856625

Coherence Score:  0.45019473548461


In [44]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.157490  0.194130       1        1  16.790598
17     0.017474 -0.235203       2        1  12.333777
15     0.131570  0.239329       3        1  11.131601
9     -0.147337 -0.218987       4        1   9.118492
7     -0.222514  0.124445       5        1   7.905663
5     -0.037648  0.237163       6        1   7.355289
0      0.250918 -0.023552       7        1   5.412356
10    -0.241430  0.003061       8        1   5.063828
1      0.117686  0.163696       9        1   4.043833
11    -0.129587 -0.060919      10        1   3.521495
19    -0.190834 -0.100053      11        1   3.386577
6     -0.040623 -0.144657      12        1   3.235037
18    -0.032933  0.090456      13        1   2.563101
3      0.104268 -0.183963      14        1   2.541469
4      0.192533 -0.102380      15        1   2.338622
13     0.173701  0.021839      16        1   1.608548
8      0.102430  0.036048      17        1   1.279417
12     0.036609 -0.013485      18        1   0.123432
14     0.036604 -0.013483      19        1   0.123432
16     0.036604 -0.013483      20        1   0.123432, topic_info=          Term       Freq      Total Category  logprob  loglift
155      think  19.000000  19.000000  Default   30.000  30.0000
91         let  11.000000  11.000000  Default   29.000  29.0000
74      option  20.000000  20.000000  Default   28.000  28.0000
51       cheap  22.000000  22.000000  Default   27.000  27.0000
27      flight  31.000000  31.000000  Default   26.000  26.0000
..         ...        ...        ...      ...      ...      ...
25   expensive   0.003936   4.204706  Topic20   -5.743  -0.2765
26       first   0.003936   6.670673  Topic20   -5.743  -0.7380
27      flight   0.003936  31.449441  Topic20   -5.743  -2.2886
28      friend   0.003936   1.484243  Topic20   -5.743   0.7648
29         get   0.003936   6.766088  Topic20   -5.743  -0.7522

[931 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
88        9  0.748789    actual
0         2  0.204441  actually
0         6  0.204441  actually
0         8  0.204441  actually
0        10  0.204441  actually
...     ...       ...       ...
176       1  0.314360      work
176       2  0.314360      work
176       8  0.314360      work
189       4  0.847956     write
132       6  0.688913      year

[638 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 18, 16, 10, 8, 6, 1, 11, 2, 12, 20, 7, 19, 4, 5, 14, 9, 13, 15, 17])

In [45]:
pyLDAvis.save_html(vis, 'PPT02.html')

# PPT03

In [46]:
all = pd.read_csv('Usability Test Data.csv')
PPT03 = all.loc[(all['Person'] == 'PPT03') & (all['Person'] != 'Interviewer')]

In [47]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT03['Question/Answer']))

print(data_words[:1])

[['the', 'last', 'trip', 'was', 'the', 'four', 'days', 'in', 'france', 'for', 'my', 'cousin', 'wedding', 'so', 'it', 'was', 'bit']]


In [48]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'last', 'trip', 'was', 'the', 'four', 'days', 'in', 'france', 'for', 'my', 'cousin', 'wedding', 'so', 'it', 'was', 'bit']


In [49]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [50]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['last', 'trip', 'day', 'cousin', 'wedding', 'bit']]


In [51]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [52]:
id2word[0]

'bit'

In [53]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('bit', 1),
  ('cousin', 1),
  ('day', 1),
  ('last', 1),
  ('trip', 1),
  ('wedding', 1)]]

In [54]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [55]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.003*"bag" + 0.003*"arrival" + 0.003*"outbound" + 0.003*"midnight" + '
  '0.003*"matter" + 0.003*"late" + 0.003*"ehe" + 0.003*"early" + 0.003*"cabin" '
  '+ 0.003*"return"'),
 (1,
  '0.033*"bit" + 0.033*"last" + 0.033*"wedding" + 0.033*"cousin" + '
  '0.033*"trip" + 0.033*"day" + 0.033*"book" + 0.033*"know" + 0.033*"strange" '
  '+ 0.033*"case"'),
 (2,
  '0.040*"ok" + 0.040*"thing" + 0.040*"never" + 0.020*"know" + 0.020*"airline" '
  '+ 0.020*"loading" + 0.020*"inaudible" + 0.020*"booking" + 0.020*"website" + '
  '0.020*"continue"'),
 (3,
  '0.043*"ok" + 0.043*"date" + 0.043*"click" + 0.043*"price" + 0.029*"think" + '
  '0.029*"outside" + 0.029*"box" + 0.015*"see" + 0.015*"show" + '
  '0.015*"inaudible"'),
 (4,
  '0.050*"sometimes" + 0.034*"give" + 0.034*"information" + 0.034*"get" + '
  '0.034*"especially" + 0.034*"step" + 0.017*"lufthansa" + 0.017*"book" + '
  '0.017*"frankfurt" + 0.017*"straight"'),
 (5,
  '0.003*"bag" + 0.003*"arrival" + 0.003*"outbound" + 0.003*"midnight"

In [56]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.111540555302239

Coherence Score:  0.47612121739874774


In [57]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.013600  0.207696       1        1  14.914395
9      0.268259 -0.122849       2        1  10.616132
8      0.232106  0.145558       3        1  10.120782
17    -0.001002 -0.262782       4        1   9.482492
19    -0.109317  0.224738       5        1   8.946651
7      0.085647  0.247837       6        1   7.949258
12     0.210689 -0.012968       7        1   7.016232
3     -0.271123  0.016816       8        1   5.295642
18     0.147774  0.128554       9        1   4.702273
4     -0.187341 -0.200670      10        1   4.381814
13    -0.193526 -0.006607      11        1   3.988451
2      0.088544 -0.192906      12        1   3.442376
16    -0.162574  0.121908      13        1   2.894154
6      0.098452 -0.047998      14        1   2.297808
11    -0.091729 -0.154997      15        1   2.002381
1     -0.008257  0.047540      16        1   1.505828
14    -0.022836 -0.034730      17        1   0.111815
15    -0.023240 -0.034418      18        1   0.111787
5     -0.023463 -0.034861      19        1   0.109866
0     -0.023463 -0.034861      20        1   0.109866, topic_info=        Term      Freq      Total Category  logprob  loglift
6    airline  14.00000  14.000000  Default    30.00  30.0000
143   browse  15.00000  15.000000  Default    29.00  29.0000
134     come   8.00000   8.000000  Default    28.00  28.0000
178    click  11.00000  11.000000  Default    27.00  27.0000
13        go  16.00000  16.000000  Default    26.00  26.0000
..       ...       ...        ...      ...      ...      ...
25      give   0.00338   5.093444  Topic20    -5.79  -0.5042
26      stop   0.00338   3.056986  Topic20    -5.79   0.0063
27      case   0.00338   3.705463  Topic20    -5.79  -0.1860
28     cheap   0.00338   7.895903  Topic20    -5.79  -0.9426
29   compare   0.00338   1.501386  Topic20    -5.79   0.7174

[974 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
265       4  0.682752     able
180      12  0.782576   accept
256       3  0.866964   accord
49        1  0.666209  account
188       4  0.320341   actual
...     ...       ...      ...
233       4  0.454492     wise
233       9  0.454492     wise
176      14  0.840328     work
295       6  0.701574    wrong
141       2  0.681968     year

[689 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 10, 9, 18, 20, 8, 13, 4, 19, 5, 14, 3, 17, 7, 12, 2, 15, 16, 6, 1])

In [58]:
pyLDAvis.save_html(vis, 'PPT03.html')